In [2]:
#imports
import pykep as pk
import numpy as np
from datetime import datetime, timedelta
import math

#Load Spice Kernel
pk.util.load_spice_kernel("/home/m/mmk40/Desktop/Planet Position/de431.bsp")

In [3]:
#Define Constants
AU = pk.AU
DAY2SEC = pk.DAY2SEC
SEC2DAY = pk.SEC2DAY
DAY2YEAR = pk.DAY2YEAR

# Define planets
earth = pk.planet.spice('EARTH', 'SUN', 'ECLIPJ2000', 'NONE')
jupiter = pk.planet.spice('JUPITER BARYCENTER', 'SUN', 'ECLIPJ2000', 'NONE')

In [4]:
# Convert spherical (RA/Dec) to Cartesian

def vector_to_ra_dec(vec):
    x, y, z = vec
    r = np.linalg.norm(vec)
    ra = np.arctan2(y, x)
    dec = np.arcsin(z / r)
    return np.degrees(ra) % 360, np.degrees(dec)


def ecliptic_to_cartesian(lambda_deg, beta_deg, distance_km):
    lam_rad = np.deg2rad(lambda_deg)
    beta_rad = np.deg2rad(beta_deg)
    x = distance_km * np.cos(beta_rad) * np.cos(lam_rad)
    y = distance_km * np.cos(beta_rad) * np.sin(lam_rad)
    z = distance_km * np.sin(beta_rad)
    return np.array([x, y, z])


In [5]:
# Define Heliopause position
lam_deg = 240
beta_deg = 7.5
heliopause_distance = 120 * AU
rH = ecliptic_to_cartesian(lam_deg, beta_deg, heliopause_distance)

In [13]:
# Earth Jupiter Lambert Leg
def compute_leg1():
    start_date = pk.epoch_from_string("2035-01-01 00:00:00")
    end_date = pk.epoch_from_string("2050-01-01 00:00:00")
    launch_step = 30
    tof_range = (300, 1500)

    results = []
    t_dep = start_date

    while t_dep.mjd2000 < end_date.mjd2000:
        rE, vE = earth.eph(t_dep)

        for tof_days in range(tof_range[0], tof_range[1], 30):
            try:
                t_arr = pk.epoch(t_dep.mjd2000 + tof_days)
                rJ, vJ = jupiter.eph(t_arr)
                dt_sec = tof_days * DAY2SEC
                l1 = pk.lambert_problem(r1=rE, r2=rJ, tof=dt_sec, mu=pk.MU_SUN, max_revs=2)

                v1 = np.array(l1.get_v1()[0])
                v2 = np.array(l1.get_v2()[0])
                v_inf_vec = (v1 - np.array(vE))
                v_inf_earth = np.linalg.norm(v_inf_vec) / 1000 # km/s # Earth Departure Speed
                C3 = v_inf_earth**2 # km2/s2

                # Calculating Fluby speed
                rp = 9 * 69911e3 # m  # radius of periapsis for the flyby trajectory (change this)
                muJ = 6.67e-11 * 1.898e27 # m3/s2  # gravitationalparameter for jupiter
                v_in = (np.linalg.norm(np.array(v2) - np.array(vJ)))/1000   # km/s Jupiter Arrival Speed

                # Calculating Turning Angle
                phi = np.degrees(np.pi - 2 * np.arccos(1 / (((v_in ** 2) * rp) / muJ + 1)))
                # print (np.pi) # Degree/radian check
                # print (f" Turning angle: {phi:.2f} degrees") # degrees
                
                
                # Jupiter Speed
                #print(vJ) # vector/scalar check
                vJ_mag = np.linalg.norm(vJ)
                #print(vJ_mag)

                
                # Law of Cosines to find the v_out
                v_inf_jupiter = math.sqrt(vJ_mag**2 + v_in**2 - 2*vJ_mag*v_in*np.cos(np.radians(phi))) / 1000 # km/s
                
                # Time to Heliopause from Jupiter flyby
                d = 194.8*AU # 200 AU in meters
                s = v_inf_jupiter*1000 # speed of space craft at jupiter exit in m/s 
                # print (d) .pk check
                time_to_200AU = (d/s)*SEC2DAY*DAY2YEAR
                total_time = time_to_200AU + tof_days/365
                flyby_speed = v_inf_jupiter - v_in

                #print(f"Time to reach Jupiter: {dt*SEC2DAY*DAY2YEAR: .2f} years")
                #print(f"Time to 200 AU from Jupiter Flyby: {time_to_200AU:.2f} years")
                #print(f"Total Time to reach 200 AU from launch: {time_to_200AU + dt*SEC2DAY*DAY2YEAR : .2f} years")

                d1 = 84.8*AU
                s1 = v_inf_jupiter*1000

                time_to_90AU = (d1/s1)*SEC2DAY*DAY2YEAR # time to reach 90 AU

                d2 = 114.8*AU
                s2 = v_inf_jupiter*1000

                time_to_120AU = (d2/s2)*SEC2DAY*DAY2YEAR # time to reach 120 AU

                if C3 <= 100 and v_inf_jupiter > 0 and total_time <= 75:
                    ra_inf, dec_inf = vector_to_ra_dec(v_inf_vec)
                    kep = pk.ic2par(rE, v1, pk.MU_SUN)
                    results.append({
                        'departure': str(t_dep),
                        'arrival': str(t_arr),
                        'tof_days': tof_days,
                        'C3': C3,
                        'delta_v': v_inf_earth,
                        'sma': kep[0],   # semi-major axis
                        'ecc': kep[1],   # eccentricity
                        'inc': np.degrees(kep[2]),  # inclination (rad → deg)
                        'raan': np.degrees(kep[3]),
                        'argp': np.degrees(kep[4]),
                        'ta': np.degrees(kep[5]),
                        'turning angle': phi,
                        'flyby speed': flyby_speed,
                        'jupiter departure speed': v_inf_jupiter,
                        'time_to_90AU': time_to_90AU,
                        'time_to_120AU': time_to_120AU,
                        'jupiter flyby to 200 au': time_to_200AU,
                        'total_time' : total_time,
                        'Outgoing_C3' : Outgoing_C3,
                        'ra_inf': ra_inf,
                        'dec_inf': dec_inf,
                    })

            except RuntimeError:
                continue

        t_dep = pk.epoch(t_dep.mjd2000 + launch_step)

    return results

In [14]:
# Jupiter to Heliopause leg

def compute_leg2(leg1_data):
    results = []
    for leg in leg1_data:
        t_dep = leg['arrival']
        rJ, vJ = jupiter.eph(t_dep)

        for tof_days in range(10000, 20000, 30):
            try:
                t_arr = pk.epoch(t_dep.mjd2000 + tof_days)
                dt_sec = tof_days * DAY2SEC
                l2 = pk.lambert_problem(r1=rJ, r2=rH, tof=dt_sec, mu=pk.MU_SUN, max_revs=2)
                v1 = np.array(l2.get_v1()[0])
                #angle = np.arccos(np.clip(np.dot(v1, leg['v_inf_vec']) / (np.linalg.norm(v1) * np.linalg.norm(leg['v_inf_vec'])), -1, 1)) * 180 / np.pi
                #if angle > 10:
                    #continue

                v_inf = (v1 - np.array(vJ)) / 1000
                C3_2 = np.linalg.norm(v_inf)**2
                if C3_2 <= 250:
                    kep = pk.ic2par(rJ, v1, pk.MU_SUN)
                    results.append({
                        't_dep': str(t_dep),
                        't_arr': str(t_arr),
                        'tof_years': tof_days / 365,
                        'C3': C3,
                        'sma': kep[0],
                        'ecc': kep[1],
                        'inc': np.degrees(kep[2]),
                        'raan': np.degrees(kep[3]),
                        'argp': np.degrees(kep[4]),
                        'ta': np.degrees(kep[5]),
                        #'angle_match': angle
                    })
            except RuntimeError:
                continue
    return results


In [15]:
# Run full mission
leg1_results = compute_leg1()
leg2_results = compute_leg2(leg1_results)

# Print results
for res in leg2_results:
    print(f"\n--- Jupiter to Heliosphere ---")
    print(f"Departure: {res['t_dep']}, Arrival: {res['t_arr']}, TOF: {res['tof_years']:.2f} years")
    print(f"C3: {res['C3']:.2f} km^2/s^2, Match Angle: {res['angle_match']:.2f}°")
    print(f"SMA: {res['sma']/1000:.2f} km, ECC: {res['ecc']:.4f}, INC: {res['inc']:.2f}°")
    print(f"RAAN: {res['raan']:.2f}°, AP: {res['argp']:.2f}°, TA: {res['ta']:.2f}°")

NameError: name 'Outgoing_C3' is not defined